In [99]:
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import musicbrainzngs

In [2]:
class MusicStream:
    def __init__(self, genero, año):
        self.genero = genero
        self.año = año
    
    def spotify(self, genero, año):
        client_id = "9ba747b4cd1a426c826e14373c7648a1"
        api_key = "4c67909da02740418998f8277a500767"

        auth_manager = SpotifyClientCredentials( client_id, api_key)
        sp = spotipy.Spotify(auth_manager=auth_manager)

        tracks ={"canciones":[], "artistas": [], "género": genero, "tipo": "track", "año lanzamiento": [], "año": año}
        for offset in range(0, 500, 50):
            canciones = sp.search(q= f"genre:{genero} year: {año}", type="track", limit=50, offset=offset)
            for track in canciones["tracks"]["items"]:
                if track not in tracks["canciones"]:
                    tracks["canciones"].append(track["name"])
                    tracks["artistas"].append(track["artists"][0]["name"])
                    try:
                        tracks["año lanzamiento"].append(track["album"]["release_date"].split("-")[0])
                    except:
                        tracks["año lanzamiento"].append("no data") 
                else: 
                    pass
        return tracks
    def musicbrainz(self, genero, año):
        tracks = self.spotify(genero, año)
        musicbrainzngs.set_useragent("MusicStream", "1.0", "psainzpc@hotmail.com")
        lista_artistas = list(set(tracks["artistas"]))  # Convertir el valor asociado a la clave 'artistas' en un set para limpiar los artistas repetidos
        artistas = {"artista": [], "pais de origen": [], "area de origen": [], "fecha inicio actividad": [], "fin de actividad": []} 
        for a in lista_artistas:
            result = musicbrainzngs.search_artists(artist=a)
            try:
                artistas["artista"].append(result["artist-list"][0]["name"])
            except:
                artistas["artista"].append("no data")
            try:
                artistas["pais de origen"].append(result["artist-list"][0]["area"]["name"])
            except:
                artistas["pais de origen"].append("no data")
            try:
                artistas["area de origen"].append(result["artist-list"][0]["begin-area"]["name"])
            except:
                artistas["area de origen"].append("no data") 
            try:
                artistas["fecha inicio actividad"].append(result["artist-list"][0]["life-span"]["begin"].split("-")[0])
            except:
                artistas["fecha inicio actividad"].append("no data")
            try:
                if result["artist-list"][0]["life-span"]["ended"] == False:
                    artistas["fin de actividad"].append("no activo")
                else:
                    artistas["fin de actividad"].append("activo")
            except:
                artistas["fin de actividad"].append("no data")
        return artistas
    def lastfm(self, genero, año):
        api_key= "cce24bdeb4a50d132f3b117a6dbf6dab"
        tracks = self.spotify(genero, año)
        lista_artistas = list(set(tracks["artistas"])) 
        artistas_lfm = {"Artista":[], "Biografia":[], "Listeners":[], "Playcount":[], "Artistas Similares":[]}
        for a in lista_artistas:
            params_info = {
                'method': 'artist.getinfo',
                'artist': a,
                'api_key': api_key,
                'format': 'json'
                }
            url = f"http://ws.audioscrobbler.com/2.0/"
            respuesta = requests.get(url, params=params_info)
            artista = respuesta.json()
            try:
                artistas_lfm['Artista'].append(artista["artist"]["name"])
            except:
                artistas_lfm["Artista"].append("no data")
            try:
                artistas_lfm["Biografia"].append(artista["artist"]["bio"]["summary"])
            except:
                artistas_lfm["Biografia"].append("no data")
            try:
                artistas_lfm["Listeners"].append(artista["artist"]["stats"]["listeners"])
            except:
                artistas_lfm["Listeners"].append("no data")
            try:
                artistas_lfm["Playcount"].append(artista["artist"]["stats"]["playcount"])
            except:
                artistas_lfm["Playcount"].append("no data")
            try:
                artistas_lfm["Artistas Similares"].append(artista["artist"]["similar"]["artist"][0]["name"])
            except:
                artistas_lfm["Artistas Similares"].append("no data")
        return artistas_lfm

In [5]:
pop = MusicStream("pop", 2019)